# mlr3：技术细节
主要介绍除了机器学习本身以外的一些设置，比如日志设置，超参数设置，并行运算等。

- 超参数设置
    - reference-based objects
    - 设定参数范围（parameter space）
- 日志控制
    - 减少或增加屏幕日志
    - 重定向日志

## 超参数设置
超参数设置是通过`paradox`包完成的。


### reference-based objects
`paradox`是`ParamHelpers`的重写版，完全基于R6对象。

In [25]:
library("paradox")

pset <- ps(
    z = p_int(lower = 1, upper = 3),
    x = p_dbl(lower = -10, upper = 10),
    flag = p_lgl(),
    methods = p_fct(c("a", "b", "c"))
)


绘制随机样本 /创建随机设计：

In [26]:
generate_design_random(pset, 3)


<Design> with 3 rows:
       z         x   flag methods
   <int>     <num> <lgcl>  <char>
1:     3 -5.921445  FALSE       c
2:     3 -7.413323  FALSE       a
3:     1 -2.293905  FALSE       a

## 生成LHS设计

In [30]:
requireNamespace("lhs")
#> Loading required namespace: lhs
generate_design_lhs(pset, 3)


Loading required namespace: lhs



ERROR: Error: The following packages could not be loaded: lhs


## Generate Grid Design:

In [28]:
generate_design_grid(pset, resolution = 2)


<Design> with 24 rows:
        z     x   flag methods
    <int> <num> <lgcl>  <char>
 1:     1   -10   TRUE       a
 2:     1   -10   TRUE       b
 3:     1   -10   TRUE       c
 4:     1   -10  FALSE       a
 5:     1   -10  FALSE       b
 6:     1   -10  FALSE       c
 7:     1    10   TRUE       a
 8:     1    10   TRUE       b
 9:     1    10   TRUE       c
10:     1    10  FALSE       a
11:     1    10  FALSE       b
12:     1    10  FALSE       c
13:     3   -10   TRUE       a
14:     3   -10   TRUE       b
15:     3   -10   TRUE       c
16:     3   -10  FALSE       a
17:     3   -10  FALSE       b
18:     3   -10  FALSE       c
19:     3    10   TRUE       a
20:     3    10   TRUE       b
21:     3    10   TRUE       c
22:     3    10  FALSE       a
23:     3    10  FALSE       b
24:     3    10  FALSE       c
        z     x   flag methods

In [31]:
pset$ids()


[1] "z"       "x"       "flag"    "methods"

In [32]:
pset$levels


$z
NULL

$x
NULL

$flag
[1]  TRUE FALSE

$methods
[1] "a" "b" "c"

In [33]:
pset$nlevels


z       x    flag methods 
      3     Inf       2       3

In [34]:
pset$is_number


z       x    flag methods 
   TRUE    TRUE   FALSE   FALSE

In [35]:
pset$lower


z       x    flag methods 
      1     -10      NA      NA

In [36]:
pset$upper


z       x    flag methods 
      3      10      NA      NA

## 参数检查
检查参数是否满足`ParamSet`的所有条件，使用`$test()`(在不匹配时返回FALSE)， `$Check()`(在不匹配时返回错误描述)和`$assert()`(在不匹配时抛出错误)

In [37]:
pset$test(list(z = 1, x = 1))


[1] TRUE

In [38]:
pset$test(list(z = -1, x = 1))


[1] FALSE

In [39]:
pset$check(list(z = -1, x = 1))


[1] "z: Element 1 is not >= 0.5"

In [40]:
pset$assert(list(z = -1, x = 1))


ERROR: Error in pset$assert(list(z = -1, x = 1)): Assertion on 'list(z = -1, x = 1)' failed: z: Element 1 is not >= 0.5.


每个参数必须包含以下4个部分：

- id - 参数名字
- default - 默认值
- special_vals - A list of values that are accepted even if they do not conform to the type
- tags - Tags that can be used to organize parameters
其中Int/Dbl还有lower/upper两个参数设定范围，Fct还有levels参数设定因子水平。

设置超参数的简便写法：

In [46]:
library(paradox)

ps <- ps(
  cp = p_dbl(lower = 0.001, upper = 0.1),
  minsplit = p_int(lower = 1, upper = 10)
)


## 日志控制

`mlr3`包做机器学习时会在屏幕输出巨多日志，有时候并不需要，我们只要结果就可以了，这个功能是通过lgr这个包实现的。

### 减少或增加屏幕日志
默认的输出日志水平是`info`，通过以下代码查看全部的日志水平：

In [47]:
getOption("lgr.log_levels")


NULL

这个水平是有顺序的，越往右边的水平输出的东西越多，所以如果你想减少mlr3包的屏幕输出内容，你可以进行如下设置：

In [48]:
lgr::get_logger("mlr3")$set_threshold("warn")


另外，`mlr3tuning`和`mlr3fselect`的日志是通过bbotk包完成的，如果想要减少其输出内容，需要进行如下设置：



In [49]:
lgr::get_logger("bbotk")$set_threshold("warn")


通过以上两句代码，就会显著减少`mlr3`包做机器学习时的屏幕输出了。

## 重定向日志
可以把日志保存到单独的JSON文件中。

In [50]:
tf <- tempfile("mlr3log_", fileext = ".json")

logger <- lgr::get_logger("mlr")

logger$add_appender(lgr::AppenderJson$new(tf), name = "json")

logger$warn("this is a warning from mlr3")


WARN  [00:56:26.184] this is a warning from mlr3


In [51]:
cat(readLines(tf))


{"level":300,"timestamp":"2024-09-02 00:56:26","logger":"mlr","caller":"eval","msg":"this is a warning from mlr3"}